In [1]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

In [3]:
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'cardiffnlp/twitter-roberta-base-sentiment',
	'HF_TASK':'text-classification'
}

In [4]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.37.0',
	pytorch_version='2.1.0',
	py_version='py310',
	env=hub,
	role=role, 
)

In [5]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type='ml.g4dn.xlarge'
)

-----------!

In [6]:
predictor.predict({
	"inputs": "I like you. I love you",
})

[{'label': 'LABEL_2', 'score': 0.9782518148422241}]

In [7]:
predictor.predict({
	"inputs": "I hate this fucking shit",
})

[{'label': 'LABEL_0', 'score': 0.979564368724823}]

In [12]:
predictor.predict({
    "inputs": "I cannot be sure without a context",
})

[{'label': 'LABEL_1', 'score': 0.6176857352256775}]